# Task 2 · German Government Yield Curve Modeling

This notebook mirrors an industry-grade workflow for building and comparing yield-curve models on German sovereign data. We will proceed incrementally so you can follow every decision:



1. **Data acquisition (today)**: Pull the latest available Bundesbank zero-coupon curve, automatically rolling back to the most recent business day and logging any gaps.

2. **Exploratory checks**: Inspect maturities, units, and coverage.

3. **Nelson–Siegel fit**: Estimate parameters with transparent initialisation and diagnostics.

4. **Cubic spline fit**: Fit a smooth curve and compare against Nelson–Siegel.

5. **Model comparison & interpretation**: Discuss fit quality, parameter meaning, and ethical considerations for smoothing.



You can execute each step as soon as you understand it; the notebook caches intermediate outputs (think: production-style audit trail) so you can revisit or rerun safely.

## Step 1 · Acquire the latest usable curve

We emulate a buy-side data pipeline:



- Start from today's date in Europe/Berlin time.

- Step backward through the German business calendar until the Bundesbank publishes the curve (weekends and public holidays are skipped automatically).

- Request zero-coupon yields for maturities 0.5Y through 30Y via the Bundesbank SDMX REST API.

- Validate that every required tenor is present and numeric.

- Persist a timestamped CSV snapshot under `data/raw/` to keep a reproducible audit trail.



> **Note**: We'll use the `holidays` package for the German calendar and let pandas handle business-day offsets. If the API is unavailable for several consecutive days, we raise a clear error so you can investigate instead of silently proceeding with stale data.

In [5]:
import sys
import subprocess
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from pathlib import Path
from io import StringIO

import numpy as np
import pandas as pd
import requests

try:
    import holidays
except ImportError:  # install on the fly for reproducibility in fresh environments
    subprocess.check_call([sys.executable, "-m", "pip", "install", "holidays", "--quiet"])
    import holidays

BASE_URL = "https://api.statistiken.bundesbank.de/rest/download/BBSIS"
DATA_CACHE_DIR = Path("data/raw")
DATA_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Residual maturity codes published by the Bundesbank (see BBSIS metadata)
# Format: RmmXX where mm corresponds to maturity in years (two digits) or 005 for 6 months
MATURITY_CODE_MAP = {
    0.5: "R005X",   # 6 months
    **{float(year): f"R{year:02d}XX" for year in range(1, 31)},
}

def build_series_key(maturity_code: str) -> str:
    return f"D.I.ZST.ZI.EUR.S1311.B.A604.{maturity_code}.R.A.A._Z._Z.A"

def create_german_calendar(start_year: int = 2010, end_year: int = 2035) -> holidays.HolidayBase:
    return holidays.Germany(years=range(start_year, end_year + 1))

def previous_business_day(current: datetime.date, holiday_calendar) -> datetime.date:
    candidate = current - timedelta(days=1)
    while candidate.weekday() >= 5 or candidate in holiday_calendar:
        candidate -= timedelta(days=1)
    return candidate

def fetch_single_series(series_key: str, start: datetime.date, end: datetime.date) -> pd.DataFrame:
    url = f"{BASE_URL}/{series_key}?format=csv&lang=en&startPeriod={start.isoformat()}&endPeriod={end.isoformat()}"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    text = response.text
    data_lines = [line for line in text.splitlines() if line and line[0].isdigit()]
    if not data_lines:
        raise ValueError(f"No datapoints returned for series {series_key} on {end}")
    df = pd.read_csv(StringIO("\n".join(data_lines)), names=["date", "value", "comment"], na_values=["."])
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d").dt.date
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df[["date", "value"]]

def extract_value_for_date(df: pd.DataFrame, as_of: datetime.date) -> float | None:
    match = df.loc[df["date"] == as_of, "value"]
    if match.empty:
        return None
    return float(match.iloc[0])

def fetch_curve_for_date(as_of_date: datetime.date) -> pd.DataFrame:
    rows = []
    missing = []
    for maturity_years, maturity_code in MATURITY_CODE_MAP.items():
        series_key = build_series_key(maturity_code)
        raw_df = fetch_single_series(series_key, as_of_date, as_of_date)
        value = extract_value_for_date(raw_df, as_of_date)
        if value is None or np.isnan(value):
            missing.append(maturity_years)
            continue
        rows.append({"maturity_years": maturity_years, "yield_pct": value})
    if missing:
        raise ValueError(f"Missing maturities for {as_of_date}: {missing}")
    curve = pd.DataFrame(rows).sort_values("maturity_years").reset_index(drop=True)
    return curve

def fetch_latest_curve(max_lookback: int = 7) -> tuple[pd.DataFrame, datetime.date]:
    tz = ZoneInfo("Europe/Berlin")
    holiday_calendar = create_german_calendar()
    candidate = datetime.now(tz).date()
    if candidate.weekday() >= 5 or candidate in holiday_calendar:
        candidate = previous_business_day(candidate, holiday_calendar)
    attempt = 0
    last_error: Exception | None = None
    while attempt <= max_lookback:
        try:
            curve = fetch_curve_for_date(candidate)
            return curve, candidate
        except Exception as exc:
            last_error = exc
            candidate = previous_business_day(candidate, holiday_calendar)
            attempt += 1
    raise RuntimeError(
        f"Unable to fetch a complete curve within {max_lookback} business days. Last error: {last_error}"
    )

def cache_curve(df: pd.DataFrame, as_of_date: datetime.date) -> Path:
    timestamp = datetime.now(ZoneInfo("Europe/Berlin")).strftime("%Y%m%dT%H%M%S")
    path = DATA_CACHE_DIR / f"bundesbank_yieldcurve_{as_of_date.isoformat()}_{timestamp}.csv"
    df.assign(as_of_date=as_of_date.isoformat()).to_csv(path, index=False)
    return path

curve_df, curve_date = fetch_latest_curve(max_lookback=10)
cache_path = cache_curve(curve_df, curve_date)

print(f"Fetched Bundesbank curve for {curve_date} with {len(curve_df)} maturities.")
print(f"Snapshot cached to {cache_path}")
curve_df

Fetched Bundesbank curve for 2025-10-07 with 31 maturities.
Snapshot cached to data/raw/bundesbank_yieldcurve_2025-10-07_20251007T160952.csv


,maturity_years,yield_pct
0,0.5,1.95
1,1.0,1.96
2,2.0,2.00
3,3.0,2.08
4,4.0,2.18
5,5.0,2.29
6,6.0,2.40
7,7.0,2.51
8,8.0,2.62
9,9.0,2.71


### What to check after running this cell

- The printed date should be the latest business day with data; note it for the report.

- Verify the DataFrame shows tenors from 0.5Y through 30Y with yields in percentage points.

- A CSV snapshot should appear under `data/raw/`; confirm the filename embeds both the curve date and fetch timestamp.



If an error appears (e.g., repeated missing maturities), increase `max_lookback` temporarily and rerun, then investigate why the API skipped multiple days.

### How each `yield_pct` is produced

1. **Map maturities to SDMX series keys.** The dictionary `MATURITY_CODE_MAP` holds every residual maturity code published by the Bundesbank (`0.5 → R005X`, `1.0 → R01XX`, …, `30.0 → R30XX`). For each entry we call `build_series_key` to embed that code into the full path `D.I.ZST.ZI.EUR.S1311.B.A604.<code>.R.A.A._Z._Z.A`.

2. **Request the day’s observations.** `fetch_single_series` downloads the CSV for that series and narrows it to rows where the first character is a digit (removing metadata blocks). Pandas reads the remaining two columns—`date` and `value`—and coerces them to `datetime.date` and `float`.

3. **Pick the exact business date.** `extract_value_for_date` filters the series to the `as_of_date` returned by `fetch_latest_curve`. If the Bundesbank skipped that tenor on the chosen date we record the maturity as missing and, after the loop, raise an error so the user can investigate.

4. **Assemble the tidy table.** Every successful lookup contributes a row `{"maturity_years": maturity, "yield_pct": value}`. Once the loop finishes a DataFrame is built from those rows and sorted by `maturity_years` before being returned as `curve_df`.

5. **Cache the result.** `cache_curve` adds an `as_of_date` column (ISO string) and writes the table to disk. The cached CSV is the same dataset we display at the end of the Step 1 cell.

In [6]:
# Display the latest curve as a maturity → yield mapping
curve_df.set_index("maturity_years")["yield_pct"]

maturity_years
0.5     1.95
1.0     1.96
2.0     2.00
3.0     2.08
4.0     2.18
5.0     2.29
6.0     2.40
7.0     2.51
8.0     2.62
9.0     2.71
10.0    2.80
11.0    2.88
12.0    2.96
13.0    3.02
14.0    3.08
15.0    3.12
16.0    3.17
17.0    3.20
18.0    3.23
19.0    3.26
20.0    3.28
21.0    3.30
22.0    3.31
23.0    3.32
24.0    3.33
25.0    3.34
26.0    3.34
27.0    3.35
28.0    3.35
29.0    3.35
30.0    3.35
Name: yield_pct, dtype: float64